In [10]:
import tensorflow as tf
import os
import re
import numpy as np
from tensorflow.python.platform import gfile


In [2]:
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string(
    'model_dir', '/Users/i550012/study/LLM/tensorflow/imagenet',
    """Path to classify_image_graph_def.pb, """
    """imagenet_synset_to_human_label_map.txt, and """
    """imagenet_2012_challenge_label_map_proto.pbtxt.""")
tf.app.flags.DEFINE_string('f', '', 'kernel')


In [3]:
class NodeLookup(object):
  """Converts integer node ID's to human readable labels."""

  def __init__(self,
               label_lookup_path=None,
               uid_lookup_path=None):
    if not label_lookup_path:
      label_lookup_path = os.path.join(
          FLAGS.model_dir, 'imagenet_2012_challenge_label_map_proto.pbtxt')
    if not uid_lookup_path:
      uid_lookup_path = os.path.join(
          FLAGS.model_dir, 'imagenet_synset_to_human_label_map.txt')
    self.node_lookup = self.load(label_lookup_path, uid_lookup_path)

  def load(self, label_lookup_path, uid_lookup_path):
    """Loads a human readable English name for each softmax node.

    Args:
      label_lookup_path: string UID to integer node ID.
      uid_lookup_path: string UID to human-readable string.

    Returns:
      dict from integer node ID to human-readable string.
    """
    if not gfile.Exists(uid_lookup_path):
      tf.logging.fatal('File does not exist %s', uid_lookup_path)
    if not gfile.Exists(label_lookup_path):
      tf.logging.fatal('File does not exist %s', label_lookup_path)

    # Loads mapping from string UID to human-readable string
    proto_as_ascii_lines = gfile.GFile(uid_lookup_path).readlines()
    uid_to_human = {}
    p = re.compile(r'[n\d]*[ \S,]*')
    for line in proto_as_ascii_lines:
      parsed_items = p.findall(line)
      uid = parsed_items[0]
      human_string = parsed_items[2]
      uid_to_human[uid] = human_string

    # Loads mapping from string UID to integer node ID.
    node_id_to_uid = {}
    proto_as_ascii = gfile.GFile(label_lookup_path).readlines()
    for line in proto_as_ascii:
      if line.startswith('  target_class:'):
        target_class = int(line.split(': ')[1])
      if line.startswith('  target_class_string:'):
        target_class_string = line.split(': ')[1]
        node_id_to_uid[target_class] = target_class_string[1:-2]

    # Loads the final mapping of integer node ID to human-readable string
    node_id_to_name = {}
    for key, val in node_id_to_uid.items():
      if val not in uid_to_human:
        tf.logging.fatal('Failed to locate: %s', val)
      name = uid_to_human[val]
      node_id_to_name[key] = name

    return node_id_to_name

  def id_to_string(self, node_id):
    if node_id not in self.node_lookup:
      return ''
    return self.node_lookup[node_id]


In [4]:
model_path = 'imagenet'
graph_path = os.path.join(model_path, 'classify_image_graph_def.pb')


In [5]:
with tf.gfile.FastGFile(graph_path, 'rb') as graphFile:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(graphFile.read())
    _ = tf.import_graph_def(graph_def, name='Imagenet')
    

Instructions for updating:
Use tf.gfile.GFile.



2024-04-18 05:28:57.863998: W tensorflow/core/framework/op_def_util.cc:357] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().


In [6]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [12]:
with tf.Session(config=config) as session:
    file_writer = tf.summary.FileWriter('logs', session.graph)
    output_tensor = session.graph.get_tensor_by_name('Imagenet/softmax:0')
    image_name = 'imagenet/cropped_panda.jpg'
    image_data = tf.gfile.FastGFile(image_name, 'rb').read()
    imput_tensor_name_data = {'Imagenet/DecodeJpeg/contents:0':image_data}
    predictions = session.run(output_tensor, imput_tensor_name_data)
    predictions = np.squeeze(predictions)
    top_predictions = predictions.argsort()[-5:][::-1]
    node_lookup = NodeLookup()
    for predict in top_predictions:
        percentage = predictions[predict]*100
        human_string = node_lookup.id_to_string(predict)
        print('%s (%.2f%%)' % (human_string, percentage))
        
            

giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca (89.11%)
indri, indris, Indri indri, Indri brevicaudatus (0.78%)
lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens (0.30%)
custard apple (0.15%)
earthstar (0.12%)
